In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
import plotly.express as px
from core.stats_utils import *
from core.sql_utils import *
import numpy as np
import pyspark.sql.functions as F
settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3 = S3Service()



## Data

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT *  FROM  vehicle v	
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join vehicle_data vd 
            on vd.vehicle_id  = v.id
            join oem o 
            on o.id = vm.oem_id
            join battery b 
            on b.id = vm.battery_id
            where o.oem_name in ('renault');"""), con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)
dbeaver_df.shape

In [ ]:
px.scatter(dbeaver_df,  x='odometer', y='soh', color="vin")

In [ ]:
px.scatter(dbeaver_df,  x='odometer', y='soh', color="net_capacity")

In [ ]:
px.scatter(dbeaver_df,  x='odometer', y='soh', color="model_name")

In [ ]:
px.scatter(dbeaver_df,  x='odometer', y='soh', color="type")

In [ ]:
vin_list = dbeaver_df[dbeaver_df["soh"] > 1.1].vin.to_list()

## Explo soh

In [ ]:
## raw_result renault

raw_results = s3.read_parquet_df_spark(spark, "result_phases/result_phases_renault.parquet").toPandas()

In [ ]:
raw_results_filter = raw_results[raw_results["VIN"].isin(vin_list)]

In [ ]:
px.scatter(raw_results_filter, x='ODOMETER_LAST', y='SOH', color='VIN')

### recompute SoH

In [ ]:
raw_results = s3.read_parquet_df_spark(spark, "raw_ts/renault/time_series/raw_ts_spark.parquet").filter(F.col("VIN").isin(vin_list)).toPandas()

In [ ]:
raw_results = raw_results.merge(dbeaver_df[['model_name', 'type', 'net_capacity', 'vin']],on="vin", how="left")